In [1]:
import os


In [2]:
import pandas as pd
import time
import pickle

In [3]:
import matgl
from matgl.ext.ase import PESCalculator, MolecularDynamics, Relaxer

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [5]:
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_std", torch.tensor(da

In [6]:
relaxer_2021_2_8_PES = Relaxer(pot)

In [7]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [8]:
def relaxer_direct_2021_2_8_PES(structure):
    return relaxer_2021_2_8_PES.relax(structure)
def relaxer_jitter_a_01_2021_2_8_PES(structure, ia = 0.01):
    structure = jitter_ia(structure, ia)
    return relaxer_2021_2_8_PES.relax(structure)#, fmax = 0.001)
def relaxer_jitter_c_005_2021_2_8_PES(structure, ic = 0.01):
    structure = jitter_ic(structure, ic)
    return relaxer_2021_2_8_PES.relax(structure)#, fmax = 0.001)

## creating a dabase 'df_calc' and running the direct relaxation in M3GNet model

In [9]:
df = structure_DB.copy()
if os.path.exists('df_calc.pkl'):
    df = pd.read_pickle('df_calc.pkl')

In [10]:
relaxer_dic_2021_2_8_PES = {'calculator_label': 'relaxer_dic_2021_2_8_PES',
                            'relaxer': relaxer_direct_2021_2_8_PES}

In [ ]:
print(df.attrs)

In [11]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-03 12:51:10 - starting packet run: attempting 10 runs of size 1

2024-09-03 12:52:03 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-03 12:52:03 - Succesfuly ran packet of 8 runs in 35.4459593296051s

53.632290840148926s


In [12]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 100, 2)
print('{}s'.format(time.time() - ti))

2024-09-03 12:52:38 - starting packet run: attempting 100 runs of size 2

2024-09-03 12:59:45 - index of ran strctures:
 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]

2024-09-03 12:59:45 - Succesfuly ran packet of 56 runs in 403.2092800140381s

427.25523948669434s


In [ ]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 100, 3)
print('{}s'.format(time.time() - ti))

2024-09-03 12:59:45 - starting packet run: attempting 100 runs of size 3



In [ ]:
df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()]

## now we run with another relaxer (using cell relaxer), but we calculate the same indeces previously calculated

In [ ]:
relaxer_dic_jitter_c_005_2021_2_8_PES = {
    'calculator_label': 'relaxer_dic_jitter_c_005_2021_2_8_PES',
    'relaxer': relaxer_jitter_c_005_2021_2_8_PES
}

In [11]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 10, 1, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-03 12:51:10 - starting packet run: attempting 10 runs of size 1

2024-09-03 12:52:03 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-03 12:52:03 - Succesfuly ran packet of 8 runs in 35.4459593296051s

53.632290840148926s


In [11]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 100, 2, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-03 12:51:10 - starting packet run: attempting 10 runs of size 1

2024-09-03 12:52:03 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-03 12:52:03 - Succesfuly ran packet of 8 runs in 35.4459593296051s

53.632290840148926s


In [11]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 100, 3, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-03 12:51:10 - starting packet run: attempting 10 runs of size 1

2024-09-03 12:52:03 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-03 12:52:03 - Succesfuly ran packet of 8 runs in 35.4459593296051s

53.632290840148926s


In [21]:
calculated_index = df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()].index


In [22]:
for i, j,k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_dic_2021_2_8_PES'].array, 
             df.loc[calculated_index, 'relaxer_dic_jitter_c_005_2021_2_8_PES'].array):
    print(compute_distortion(i, j[0]))
    print(compute_distortion(i, k[0]))
    print()

0.08171670991394009
0.08166277357433269

0.022529365287150514
0.02322317991696847

0.01929429646574994
0.02078546657726485

0.0067257354750165105
0.008730287298227519

0.04627730708397882
0.040753538829881135

